<a href="https://colab.research.google.com/github/kiarashgh98/Topic-modeling-/blob/main/1_preprocessing_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Imports and Downloads 📢**

##### Import Packages

In [ ]:
!pip install spacy_download

In [ ]:
# Standard libraries
import re
import string
from collections import defaultdict

# Data manipulation and analysis
import pandas as pd

# Text processing and analysis
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from spacy_download import load_spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

# Modeling and analysis
from gensim.models import Phrases
from gensim.models.phrases import Phraser

##### Download NLTK resources

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **Dataset 🗂️**

### Open CSV File

In [ ]:
# Using on_bad_lines (pandas 1.3.0 and above)
org_df = pd.read_csv('/content/drive/MyDrive/Kiarash/New Codes/scopus.csv')
df = org_df

# **Pre-Processing 🧰**

##### Fill missing values in all columns of Dataframe


In [ ]:
df = df.fillna('')

##### Concatenate 'Article Title', 'Author Keywords', and 'Abstract' for each paper

In [ ]:
df

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,Publisher,Combined_Text
0,Schilling-Vacaflor A.; Gustafsson M.-T.,"Schilling-Vacaflor, Almut (25622806800); Gusta...",25622806800; 56650651900,Integrating human rights in the sustainability...,2024,Environmental Science and Policy,https://www.scopus.com/inward/record.uri?eid=2...,"Chair of International Business, Society and S...","Schilling-Vacaflor A., Chair of International ...","In contemporary discourse, the need to address...",Brazil; Environmental governance; Europe; Poli...,Elsevier Ltd,Integrating human rights in the sustainability...
1,Alegoz M.; Karaer Ö.,"Alegoz, Mehmet (56440183900); Karaer, Özgen (2...",56440183900; 23009017400,Compound effect of closing the loop and servit...,2024,International Journal of Production Economics,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Industrial Engineering, Eskisehi...","Alegoz M., Department of Industrial Engineerin...",Companies generally strive to become more sust...,Closed-loop supply chain; Selling; Servitizati...,Elsevier B.V.,Compound effect of closing the loop and servit...
2,Shubailat O.M.; Al-Zaqeba M.A.A.; Madi A.; Aba...,"Shubailat, Omar M. (55173652900); Al-Zaqeba, M...",55173652900; 57216440210; 57190396548; 5720923...,Customs intelligence and risk management in su...,2024,Uncertain Supply Chain Management,https://www.scopus.com/inward/record.uri?eid=2...,"Business School, German Jordanian University, ...","Shubailat O.M., Business School, German Jordan...",In this descriptive analytical study conducted...,Customs Intelligence; Customs Regulations; Dat...,Growing Science,Customs intelligence and risk management in su...
3,Untari D.T.; Satria B.,"Untari, Dhian Tyas (57200115567); Satria, Budi...",57200115567; 57830887900,The relationship between supply chain manageme...,2024,Uncertain Supply Chain Management,https://www.scopus.com/inward/record.uri?eid=2...,"Bhayangkara Jakarta Raya University, Indonesia...","Untari D.T., Bhayangkara Jakarta Raya Universi...",Betawi Ora is an indigenous tribe that lives i...,Betawi Ora; Eco-culinary Product; Environmenta...,Growing Science,The relationship between supply chain manageme...
4,Wiyono A.; Aziz M.; Sholehudin A.; Sukrawan Y....,"Wiyono, Apri (57200074311); Aziz, Muhammad (56...",57200074311; 56436934500; 58903039800; 5720502...,Syngas optimization and tar reduction via mult...,2024,South African Journal of Chemical Engineering,https://www.scopus.com/inward/record.uri?eid=2...,Department of Automotive Engineering Education...,"Wiyono A., Department of Automotive Engineerin...",Biomass stands out as a promising availability...,Catalyst pore; Catalyzer; Energy; Multistage c...,Elsevier B.V.,Syngas optimization and tar reduction via mult...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16899,Vokurka R.J.; Zank G.M.; Lund C.M.,"Vokurka, Robert J. (7003301962); Zank, Gail M....",7003301962; 7005913203; 56789613600,Improving competitiveness through supply chain...,2002,Competitiveness Review,https://www.scopus.com/inward/record.uri?eid=2...,"Texas A&M University, Corpus Christi, United S...","Vokurka R.J., Texas A&M University, Corpus Chr...",Supply chains can improve their performance by...,,Emerald Group Publishing Ltd.,Improving competitiveness through supply chain...
16900,Bon R.; Hutchinson K.,"Bon, Ranko (56188046500); Hutchinson, Keith (8...",56188046500; 8444651100,Sustainable construction: Some economic challe...,2000,Building Research and Information,https://www.scopus.com/inward/record.uri?eid=2...,Department of Construction Management and Engi...,"Bon R., Department of Construction Management ...",Sustainable construction faces economic challe...,Economics; Environment; Policy tools; Regulati...,Routledge,Sustainable construction: Some economic challe...
16901,Leiper Q.J.; Riley P.; Uren S.,"Leiper, Q.J. (6603343238); Riley, P. (58334743...",6603343238;

In [ ]:
df['Combined_Text'] = df['Title'] + ' ' + df['Author Keywords'] + ' ' + df['Abstract']

##### Convert all words to lowercase and remove punctuation, whitespaces, email addresses, and URLs

In [ ]:
df['Combined_Text'] = df['Combined_Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))
df['Combined_Text'] = df['Combined_Text'].apply(lambda x: re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', x))
df['Combined_Text'] = df['Combined_Text'].apply(lambda x: re.sub(r'(http|https)://[a-zA-Z0-9./?%&=]+', '', x))

##### Remove stop words, keywords, misread words, and other non-semantic contributing words "BEFORE LEMMETIZATION"

In [ ]:
# Define additional_stop_words
additional_stop_words = ['from', 'subject', 're', 'edu', 'use', 'also', 'first', 'may', 'want', 'data', 'many', 'however',
                         'within', 'paper', 'research', 'although', 'across', 'based', 'conclusion', 'conclude', 'one',
                         'the', 'method', 'show', 'among', 'two', 'due', 'we', 'this', 'in', 'project',
                         'methodological', 'finding', 'new', 'large' , 'min' , ' max' , 'three' , 'set' ,
                         'key' , 'take' , 'sc' , 'help' , 'main','model','case','right','find','show','goal','ltd',
                         'low','high','factor', 'sustainability','study', 'supplychain', 'supplying', 'supplychains',
                         'environmental', 'supplier', 'performance', 'product', 'production', 'industry', 'firm',
                         'practice','system', 'supply', 'chain', 'chains' 'sustainable', 'supplied', 'supplier', 'suppliers', 'supplies',
                         'management','â', 'analysis', 'impact', 'result', 'provide', 'propose', 'approach', 'develop', 'development']


# Remove stop words, misread words, and other non-semantic contributing words
stop_words = stopwords.words('english')
stop_words.extend(additional_stop_words)
df['cleaned_text'] = df['Combined_Text'].apply(lambda text: ' '.join(word for word in word_tokenize(text) if word.isalpha() and word not in stop_words))

##### Detect and create bigrams

In [ ]:
sentences = [word_tokenize(text) for text in df['cleaned_text']]
bigram = Phrases(sentences, min_count=5, threshold=100)  # Adjust min_count and threshold as needed
bigram_mod = Phraser(bigram)

df['cleaned_text'] = df['cleaned_text'].apply(lambda text: ' '.join(bigram_mod[word_tokenize(text)]))

##### Lemmatize words to their root form

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for word, pos_tag in pos_tag(word_tokenize(text))]))

##### Remove stop words, keywords, misread words, and other non-semantic contributing words "AFTER LEMMETIZATION"

In [ ]:
# Define additional_stop_words
additional_stop_words = ['from', 'subject', 're', 'edu', 'use', 'also', 'first', 'may', 'want', 'data', 'many', 'however',
                         'within', 'paper', 'research', 'although', 'across', 'based', 'conclusion', 'conclude', 'one',
                         'the', 'method', 'show', 'among', 'two', 'due', 'we', 'this', 'in', 'project',
                         'methodological', 'finding', 'new', 'large' , 'min' , ' max' , 'three' , 'set' ,
                         'key' , 'take' , 'sc' , 'help' , 'main','model','case','right','find','show','goal','ltd',
                         'low','high','factor', 'sustainability','study', 'supplychain', 'supplying', 'supplychains',
                         'environmental', 'supplier', 'performance', 'product', 'production', 'industry', 'firm',
                         'practice','system', 'supply', 'chain', 'chains' 'sustainable', 'supplied', 'supplier', 'suppliers', 'supplies',
                         'management','â', 'analysis', 'impact', 'result', 'provide', 'propose', 'approach', 'develop', 'development']



df['cleaned_text'] = df['cleaned_text'].apply(lambda text: ' '.join(word for word in word_tokenize(text) if word not in additional_stop_words))

##### Remove infrequent words


In [ ]:
word_frequency = df['cleaned_text'].str.split(expand=True).stack().value_counts()
infrequent_words = word_frequency[word_frequency < 2].index
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: ' '.join([word for word in word_tokenize(text) if word not in infrequent_words]))

In [ ]:
df.to_csv('/content/drive/MyDrive/Kiarash/New Codes/mahsa_data_after_preprocessing.csv')